In [1]:
import os
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np 
from scipy.signal import welch
from detect_peaks import detect_peaks

In [2]:
HomeDirectory = "/Users/tbryan/Desktop/9 2019 Fall/ECEN 403/Programming/Programs_October13"
os.chdir(HomeDirectory)
print(os.getcwd())

/Users/tbryan/Desktop/9 2019 Fall/ECEN 403/Programming/Programs_October13


In [3]:
os.chdir('Data')
os.chdir('IMS')
directory = os.listdir('1st_test')
os.chdir('1st_test')
print(directory[1])

2003.11.17.23.42.30


In [4]:
data = pd.read_table(directory[0],header = None)
data.columns = ['b1x','b1y','b2x','b2y','b3x','b3y','b4x','b4y']
print(data.head())

     b1x    b1y    b2x    b2y    b3x    b3y    b4x    b4y
0 -0.146 -0.073 -0.168 -0.120 -0.024 -0.022 -0.063 -0.193
1 -0.081 -0.110 -0.107 -0.173 -0.198 -0.151  0.002 -0.125
2 -0.110  0.000 -0.151 -0.234  0.034  0.127  0.034 -0.176
3 -0.269 -0.002 -0.144 -0.212  0.007 -0.051 -0.066 -0.122
4 -0.200 -0.103 -0.215 -0.227 -0.142  0.061 -0.103 -0.059


In [5]:
b1x = np.transpose(data.values[:,0])
b1y = np.transpose(data.values[:,1])
b2x = np.transpose(data.values[:,2])
b2y = np.transpose(data.values[:,3])
b3x = np.transpose(data.values[:,4])
b3y = np.transpose(data.values[:,5])
b4x = np.transpose(data.values[:,6])
b4y = np.transpose(data.values[:,7])
x = data.index.values
print(x)

[    0     1     2 ... 20477 20478 20479]


In [6]:
"""
n: Shaft rotational speed [Hz] 2000 rpm
N: No. of rolling elements [-] 16
Bd: Diameter of a rolling element [mm] 0.331 in
Pd: Pitch diameter [mm] 2.815 in
ϕ: Contact angle [rad] 15.17*pi/180
"""

n = 2000 / 60
N = 16
Bd = 0.331*254
Pd = 2.815*254
phi = 15.17 * np.pi / 180


In [7]:
def BearingInfomation(n,N,Bd,Pd,phi):
    xx = Bd/Pd*np.cos(phi)
    BPFI = (N/2)*(1 + xx)*n
    BPFO = (N/2)*(1 - xx)*n
    BSF = (Pd/(2*Bd))*(1-(xx)**2)*n
    FTF= (1/2)*(1 - xx)*n
    x = {
        "BPFI": BPFI,
        "BPFO": BPFO,
        "BSF":  BSF,
        "FTF":  FTF
    }
    return x

In [8]:
def RemoveDCOffset(sig):
    m = sig - np.mean(sig)
    return m

In [9]:
def FourierTransform(comb_sig, T, N, f_s):
    #Fast Fourier Transform
    #number_of_time_samples = len(t)
    number_of_time_samples = N
    frq = np.arange(number_of_time_samples)/(Tmax)# two sides frequency range
    frq = frq[range(int(number_of_time_samples/(2)))] # one side frequency range
    Y = abs(np.fft.fft(comb_sig))/number_of_time_samples # fft computing and normalization
    Y = Y[range(int(number_of_time_samples/2))]
    #End fft
    x = {
        "Frequency":frq,
        "Freq. Amp.": Y
        }
    return x

In [10]:
def get_psd_values(comb_sig, T, N, fs):
    frq, psd_values = welch(comb_sig, fs=fs)
    x = {
        "Frequency":frq,
        "PSD": psd_values
        }
    return x

In [11]:
def autocorr(x):
    result = np.correlate(x, x, mode='full')
    return result[len(result)//2:]
 
def get_autocorr_values(comb_sig, T, N, f_s):
    autocorr_values = autocorr(comb_sig)
    x_values = np.array([T * jj for jj in range(0, N)])
    x = {
        "X Values":x_values,
        "Autocorr Values": autocorr_values
        }
    return x

In [12]:
from scipy.stats import kurtosis
from scipy.stats import skew
from scipy.signal import hilbert

def TimeDomainInformation(comb_sig, T, N, f_s):
    x = {
        "RMS": np.mean(comb_sig**2),
        "Mean": np.mean(comb_sig),
        "Max": np.max(comb_sig),
        "Min": np.min(comb_sig),
        "Kurtosis": kurtosis(comb_sig),
        "Skew": skew(comb_sig)
    }

    return x

In [13]:
def GetSortedPeak(frq,comb_sig, T, N, f_s):
    max_peak_height = 0.1 * np.nanmax(comb_sig)
    threshold = 0.05 * np.nanmax(comb_sig)
    #Get indices of peak
    peak = detect_peaks(comb_sig,edge = 'rising',mph = max_peak_height, mpd = 2, threshold = threshold )
    
    m = []
    mm = []
    for i in peak:
        m.append(comb_sig[i]) 
        mm.append(frq[i])

    mmm = np.argsort(m)
    n = []
    nn = []
    for i in mmm:
        n.append(m[i])
        nn.append(mm[i])

    n  = n[::-1]
    nn = nn[::-1]

    return n, nn

In [14]:
def FrequencyDomainInformation(comb_sig, T, N, f_s):
    x1 = FourierTransform(comb_sig, T, N, f_s)
    x2 = get_psd_values(comb_sig, T, N, f_s)
    x3 = get_autocorr_values(comb_sig, T, N, f_s)
    FTamp,FTfreq = GetSortedPeak(x1['Frequency'],x1['Freq. Amp.'], T, N, f_s)
    PSDamp,PSDfreq = GetSortedPeak(x2['Frequency'],x2['PSD'], T, N, f_s)
    Cor,CorTime = GetSortedPeak(x3['X Values'],x3['Autocorr Values'], T, N, f_s)
    
    while len(FTamp) <= 5:
        FTamp.append([False])
    while len(FTfreq) <= 5:
        FTfreq.append([False])
    while len(PSDamp) <= 5:
        PSDamp.append([False])
    while len(PSDfreq) <= 5:
        PSDfreq.append([False])
    while len(Cor) <= 5:
        Cor.append([False])
    while len(CorTime) <= 5:
        CorTime.append([False])
    
    x = {
        "FFT Frq @ Peak 1": FTfreq[0],
        "FFT Frq @ Peak 2": FTfreq[1],
        "FFT Frq @ Peak 3": FTfreq[2],
        "FFT Frq @ Peak 4": FTfreq[3],
        "FFT Frq @ Peak 5": FTfreq[4],
        "FFT Amp @ Peak 1": FTamp[0],
        "FFT Amp @ Peak 2": FTamp[1],
        "FFT Amp @ Peak 3": FTamp[2],
        "FFT Amp @ Peak 4": FTamp[3],
        "FFT Amp @ Peak 5": FTamp[4],
        "PSD Frq @ Peak 1": PSDfreq[0],
        "PSD Frq @ Peak 2": PSDfreq[1],
        "PSD Frq @ Peak 3": PSDfreq[2],
        "PSD Frq @ Peak 4": PSDfreq[3],
        "PSD Frq @ Peak 5": PSDfreq[4],
        "PSD Amp @ Peak 1": PSDamp[0],
        "PSD Amp @ Peak 2": PSDamp[1],
        "PSD Amp @ Peak 3": PSDamp[2],
        "PSD Amp @ Peak 4": PSDamp[3],
        "PSD Amp @ Peak 5": PSDamp[4],
        "Autocorrelate Time @ Peak 1": CorTime[0],
        "Autocorrelate Time @ Peak 2": CorTime[1],
        "Autocorrelate Time @ Peak 3": CorTime[2],
        "Autocorrelate Time @ Peak 4": CorTime[3],
        "Autocorrelate Time @ Peak 5": CorTime[4],
        "Autocorrelate @ Peak 1": Cor[0],
        "Autocorrelate @ Peak 2": Cor[1],
        "Autocorrelate @ Peak 3": Cor[2],
        "Autocorrelate @ Peak 4": Cor[3],
        "Autocorrelate @ Peak 5": Cor[4]
    }
    return x

In [15]:
def StateInformation(comb_sig, T, N, f_s):
    x = {
        "State": "Good"
    }
    return x

In [16]:
def MotorInformation(comb_sig, T, N, f_s):
    x = {
        "Motor Type AC(1)-DC(0)": 1,
        "Shaft Speed [Hz]": 2000/60
    }
    return x

In [19]:
SampleFrequency = 20000
NumberOfSamples = len(data.index.values)
dt = 1/SampleFrequency
Tmax = dt*NumberOfSamples
t = np.arange(0,Tmax,dt) #same as x*dt


In [20]:
sig = RemoveDCOffset(sig)
BearingInfo = BearingInfomation(n,N,Bd,Pd,phi)
TimeDomainInfo = TimeDomainInformation(sig,Tmax,NumberOfSamples,SampleFrequency)
FrequecyDomainInfo = FrequencyDomainInformation(sig,Tmax,NumberOfSamples,SampleFrequency)
StateInfo = StateInformation(sig,Tmax,NumberOfSamples,SampleFrequency)
MotorInfo = MotorInformation(sig,Tmax,NumberOfSamples,SampleFrequency)

In [21]:
Features = {**StateInfo,**MotorInfo,**BearingInfo,**TimeDomainInfo,**FrequecyDomainInfo}
Features = pd.DataFrame(Features, index=[0])

print(Features)

  State  Motor Type AC(1)-DC(0)  Shaft Speed [Hz]        BPFI        BPFO  \
0  Good                       1         33.333333  296.929862  236.403471   

          BSF        FTF       RMS          Mean       Max  \
0  139.916656  14.775217  0.008119  8.673617e-19  0.429092   

            ...            Autocorrelate Time @ Peak 1  \
0           ...                                 22.528   

   Autocorrelate Time @ Peak 2  Autocorrelate Time @ Peak 3  \
0                       18.432                        False   

   Autocorrelate Time @ Peak 4  Autocorrelate Time @ Peak 5  \
0                        False                        False   

   Autocorrelate @ Peak 1  Autocorrelate @ Peak 2  Autocorrelate @ Peak 3  \
0               65.668293               47.454991                   False   

   Autocorrelate @ Peak 4  Autocorrelate @ Peak 5  
0                   False                   False  

[1 rows x 43 columns]


In [26]:
ColumnTitle = Features.columns
Data = Features.values[0,:]
m = []
for file in directory:
    m.append(Data)

Data = pd.DataFrame(m,columns = ColumnTitle)
print(Data)

     State  Motor Type AC(1)-DC(0)  Shaft Speed [Hz]        BPFI        BPFO  \
0     Good                       1         33.333333  296.929862  236.403471   
1     Good                       1         33.333333  296.929862  236.403471   
2     Good                       1         33.333333  296.929862  236.403471   
3     Good                       1         33.333333  296.929862  236.403471   
4     Good                       1         33.333333  296.929862  236.403471   
5     Good                       1         33.333333  296.929862  236.403471   
6     Good                       1         33.333333  296.929862  236.403471   
7     Good                       1         33.333333  296.929862  236.403471   
8     Good                       1         33.333333  296.929862  236.403471   
9     Good                       1         33.333333  296.929862  236.403471   
10    Good                       1         33.333333  296.929862  236.403471   
11    Good                       1      